## CENSUS DATABASE 2011-2018

In [62]:
import pandas as pd
from census import Census
import requests
import os
import numpy as np

# Census API Key
from config import api_key

# provide the api key and the year to establish a session
c = Census(api_key, year=2015)

# Set an option to allow up to 300 characters to print in each column
pd.set_option('max_colwidth', 300)

In [63]:
# query for all tables
tables = c.acs5.tables()

# The tables variable contains a list of dicts, so we can convert directly to a dataframe
table_df = pd.DataFrame(tables)
print(f"Number of available tables: {len(table_df)}")
table_df.head()

Number of available tables: 1045


,description,name,variables
0,POVERTY STATUS IN THE PAST 12 MONTHS OF FAMILIES BY FAMILY TYPE BY SOCIAL SECURITY INCOME BY SUPPLEMENTAL SECURITY INCOME (SSI) AND CASH PUBLIC ASSISTANCE INCOME,B17015,https://api.census.gov/data/2015/acs/acs5/groups/B17015.json
1,SEX BY AGE BY COGNITIVE DIFFICULTY,B18104,https://api.census.gov/data/2015/acs/acs5/groups/B18104.json
2,POVERTY STATUS IN THE PAST 12 MONTHS OF FAMILIES BY FAMILY TYPE BY WORK EXPERIENCE OF HOUSEHOLDER AND SPOUSE,B17016,https://api.census.gov/data/2015/acs/acs5/groups/B17016.json
3,SEX BY AGE BY AMBULATORY DIFFICULTY,B18105,https://api.census.gov/data/2015/acs/acs5/groups/B18105.json
4,POVERTY STATUS IN THE PAST 12 MONTHS BY HOUSEHOLD TYPE BY AGE OF HOUSEHOLDER,B17017,https://api.census.gov/data/2015/acs/acs5/groups/B17017.json


In [64]:
# Filter database : Income
table_df[table_df['description'].str.contains("FAMILY")]

,description,name,variables
0,POVERTY STATUS IN THE PAST 12 MONTHS OF FAMILIES BY FAMILY TYPE BY SOCIAL SECURITY INCOME BY SUPPLEMENTAL SECURITY INCOME (SSI) AND CASH PUBLIC ASSISTANCE INCOME,B17015,https://api.census.gov/data/2015/acs/acs5/groups/B17015.json
2,POVERTY STATUS IN THE PAST 12 MONTHS OF FAMILIES BY FAMILY TYPE BY WORK EXPERIENCE OF HOUSEHOLDER AND SPOUSE,B17016,https://api.census.gov/data/2015/acs/acs5/groups/B17016.json
8,AGGREGATE INCOME DEFICIT (DOLLARS) IN THE PAST 12 MONTHS FOR FAMILIES BY FAMILY TYPE,B17011,https://api.census.gov/data/2015/acs/acs5/groups/B17011.json
13,POVERTY STATUS IN THE PAST 12 MONTHS OF FAMILIES BY HOUSEHOLD TYPE BY NUMBER OF PERSONS IN FAMILY,B17013,https://api.census.gov/data/2015/acs/acs5/groups/B17013.json
15,POVERTY STATUS IN THE PAST 12 MONTHS OF FAMILIES BY HOUSEHOLD TYPE BY NUMBER OF WORKERS IN FAMILY,B17014,https://api.census.gov/data/2015/acs/acs5/groups/B17014.json
20,POVERTY STATUS IN THE PAST 12 MONTHS OF FAMILIES BY FAMILY TYPE BY PRESENCE OF RELATED CHILDREN UNDER 18 YEARS BY AGE OF RELATED CHILDREN,B17010,https://api.census.gov/data/2015/acs/acs5/groups/B17010.json
46,POVERTY STATUS IN THE PAST 12 MONTHS OF RELATED CHILDREN UNDER 18 YEARS BY FAMILY TYPE BY AGE OF RELATED CHILDREN UNDER 18 YEARS,B17006,https://api.census.gov/data/2015/acs/acs5/groups/B17006.json
171,AGGREGATE NONFAMILY HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2015 INFLATION-ADJUSTED DOLLARS) BY SEX OF HOUSEHOLDER BY LIVING ALONE BY AGE OF HOUSEHOLDER,B19216,https://api.census.gov/data/2015/acs/acs5/groups/B19216.json
172,MEDIAN NONFAMILY HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2015 INFLATION-ADJUSTED DOLLARS) BY SEX OF HOUSEHOLDER BY LIVING ALONE BY AGE OF HOUSEHOLDER,B19215,https://api.census.gov/data/2015/acs/acs5/groups/B19215.json
173,AGGREGATE NONFAMILY HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2015 INFLATION-ADJUSTED DOLLARS),B19214,https://api.census.gov/data/2015/acs/acs5/groups/B19214.json


In [65]:
# Determine which table you're interested in
table_id = 'B19119'

# Capture the variables URL from the table_df
url = table_df.loc[table_df['name']==table_id, 'variables'].values[0]

# Make the API call
response = requests.get(url).json()

# convert the response to a DataFrame
variables = pd.DataFrame(response['variables']).transpose()

print(f"Number of available variables: {len(variables)}")
variables.head(15)

Number of available variables: 28


,concept,group,label,limit,predicateOnly,predicateType
B19119_007MA,NaN,B19119,Annotation of Margin of Error!!Total!!7-or-more-person families,0,True,string
B19119_007EA,NaN,B19119,Annotation of Estimate!!Total!!7-or-more-person families,0,True,string
B19119_006EA,NaN,B19119,Annotation of Estimate!!Total!!6-person families,0,True,string
B19119_006MA,NaN,B19119,Annotation of Margin of Error!!Total!!6-person families,0,True,string
B19119_001MA,NaN,B19119,Annotation of Margin of Error!!Total,0,True,string
B19119_001EA,NaN,B19119,Annotation of Estimate!!Total,0,True,string
B19119_005MA,NaN,B19119,Annotation of Margin of Error!!Total!!5-person families,0,True,string
B19119_005EA,NaN,B19119,Annotation of Estimate!!Total!!5-person families,0,True,string
B19119_004MA,NaN,B19119,Annotation of Margin of Error!!Total!!4-person families,0,True,string
B19119_004EA,NaN,B19119,Annotation of Estimate!!Total!!4-person families,0,True,string


In [66]:
variables[variables['predicateType']=='int'].head()

,concept,group,label,limit,predicateOnly,predicateType
B19119_005E,MEDIAN FAMILY INCOME IN THE PAST 12 MONTHS (IN 2015 INFLATION-ADJUSTED DOLLARS) BY FAMILY SIZE,B19119,Estimate!!Total!!5-person families,0,True,int
B19119_005M,MEDIAN FAMILY INCOME IN THE PAST 12 MONTHS (IN 2015 INFLATION-ADJUSTED DOLLARS) BY FAMILY SIZE,B19119,Margin of Error!!Total!!5-person families,0,True,int
B19119_006E,MEDIAN FAMILY INCOME IN THE PAST 12 MONTHS (IN 2015 INFLATION-ADJUSTED DOLLARS) BY FAMILY SIZE,B19119,Estimate!!Total!!6-person families,0,True,int
B19119_006M,MEDIAN FAMILY INCOME IN THE PAST 12 MONTHS (IN 2015 INFLATION-ADJUSTED DOLLARS) BY FAMILY SIZE,B19119,Margin of Error!!Total!!6-person families,0,True,int
B19119_007E,MEDIAN FAMILY INCOME IN THE PAST 12 MONTHS (IN 2015 INFLATION-ADJUSTED DOLLARS) BY FAMILY SIZE,B19119,Estimate!!Total!!7-or-more-person families,0,True,int


In [67]:
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E", "B19301_001E"), 
                         {'for': 'zip code tabulation area:*'})

census_data[:5]

[{'NAME': 'ZCTA5 12810',
  'B19013_001E': 57500.0,
  'B01003_001E': 724.0,
  'B01002_001E': 46.5,
  'B19301_001E': 25551.0,
  'zip code tabulation area': '12810'},
 {'NAME': 'ZCTA5 12811',
  'B19013_001E': -666666666.0,
  'B01003_001E': 67.0,
  'B01002_001E': 45.9,
  'B19301_001E': 11590.0,
  'zip code tabulation area': '12811'},
 {'NAME': 'ZCTA5 12812',
  'B19013_001E': 49583.0,
  'B01003_001E': 58.0,
  'B01002_001E': 64.1,
  'B19301_001E': 23600.0,
  'zip code tabulation area': '12812'},
 {'NAME': 'ZCTA5 12814',
  'B19013_001E': 58176.0,
  'B01003_001E': 1282.0,
  'B01002_001E': 46.4,
  'B19301_001E': 35508.0,
  'zip code tabulation area': '12814'},
 {'NAME': 'ZCTA5 12815',
  'B19013_001E': 60458.0,
  'B01003_001E': 1103.0,
  'B01002_001E': 54.2,
  'B19301_001E': 30685.0,
  'zip code tabulation area': '12815'}]

In [68]:
# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Renaming columns to be more user-friendly
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "NAME": "Name", 
                                      "zip code tabulation area": "Zipcode"})

# Since Census doesn't provide the poverty rate, we can divide Poverty Count by Population to calculate it ourselves
#census_pd["Poverty Rate"] = 100 * census_pd["Poverty Count"].astype(int) / census_pd["Population"].astype(int)

# Reorder columns and only include ones we're interested in for the final DataFrame
census_pd = census_pd[["Zipcode", "Population", "Median Age", "Household Income",
                       "Per Capita Income"]]

# Visualize
# print("Total number of zip codes in response: " + str(len(census_pd)))
census_pd.head(50)

,Zipcode,Population,Median Age,Household Income,Per Capita Income
0,12810,724.0,46.5,57500.0,25551.0
1,12811,67.0,45.9,-666666666.0,11590.0
2,12812,58.0,64.1,49583.0,23600.0
3,12814,1282.0,46.4,58176.0,35508.0
4,12815,1103.0,54.2,60458.0,30685.0
5,12816,4417.0,46.9,50417.0,26988.0
6,12817,2330.0,45.2,59861.0,26976.0
7,12819,309.0,55.6,43839.0,23297.0
8,12821,2816.0,35.6,34000.0,3450.0
9,12822,6277.0,40.4,52578.0,24178.0


In [69]:
census_pd['Zipcode'] = census_pd['Zipcode'].astype('str')
census_pd.to_csv("census_data_2015.csv", encoding="utf-8", index=False)

## EXPORTED CSV FROM CENSUS DATABASE FORMATTING

In [96]:
# Load csv file

census_2015 = os.path.join("data", "census_data_2015.csv")
census_2016 = os.path.join("data", "census_data_2016.csv")
census_2017 = os.path.join("data", "census_data_2017.csv")


census_2015_pd = pd.read_csv(census_2015, dtype ='str')
census_2016_pd = pd.read_csv(census_2016, dtype ='str')
census_2017_pd = pd.read_csv(census_2017, dtype ='str')


### This section is for cleaning the csv. Make sure you select the year you desire to format.

#### Check if there are null values in the data

In [97]:
census_2017_pd.isnull().any()

Zipcode              False
Population           False
Median Age           False
Household Income     False
Per Capita Income     True
dtype: bool

#### There is a -666666666.0 value that needs to be replaced with NaN

In [98]:

census_2017_pd_clean = census_2017_pd.replace('-666666666.0', np.nan)
census_2017_pd_clean

,Zipcode,Population,Median Age,Household Income,Per Capita Income
0,05762,513.0,39.2,58558.0,24235.0
1,05763,2715.0,43.9,54968.0,29674.0
2,05764,3370.0,41.3,44468.0,22651.0
3,05765,1632.0,46.7,56771.0,30786.0
4,05766,598.0,45.9,52250.0,33227.0
5,05767,1053.0,50.2,46193.0,30351.0
6,05769,1192.0,45.2,57500.0,26407.0
7,05770,1038.0,49.6,71667.0,34765.0
8,05772,656.0,50.0,54271.0,33247.0
9,05773,2281.0,47.8,51942.0,29896.0


#### Drop NaN values from the dataframe

In [99]:
census_2017_pd_nonan = census_2017_pd_clean.dropna(how='any')

In [100]:
census_2017_pd_nonan

,Zipcode,Population,Median Age,Household Income,Per Capita Income
0,05762,513.0,39.2,58558.0,24235.0
1,05763,2715.0,43.9,54968.0,29674.0
2,05764,3370.0,41.3,44468.0,22651.0
3,05765,1632.0,46.7,56771.0,30786.0
4,05766,598.0,45.9,52250.0,33227.0
5,05767,1053.0,50.2,46193.0,30351.0
6,05769,1192.0,45.2,57500.0,26407.0
7,05770,1038.0,49.6,71667.0,34765.0
8,05772,656.0,50.0,54271.0,33247.0
9,05773,2281.0,47.8,51942.0,29896.0


#### Export csv

In [101]:
census_2017_pd_nonan.to_csv("census_data_2017_clean.csv", encoding="utf-8", index=False)